<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Tercer Parcial</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M.en.C. Miguel Angel Pérez León</i></h5>
  <h5 align="center"><i>Ayudante: Jesús Iván Calderón Coss</i></h5>
  <h5 align="center"><i>Materia: Análisis Numérico</i></h5>
  </font>

DERECK ARIEL MIGUEL MEDRANO
318149875

# (17 Puntos + Puntos Extra + S. E.) Cuarto Parcial

El examen consiste en elegir las preguntas que deseas responder para aprobar (en el mejor de los casos el examen completo). Con respecto a las secciones teóricas, es necesario contestarlas con texto y lenguaje matemático, mientras que para las secciones prácticas es obligatorio escribir el código en *python* para realizar las pruebas respectivas.

Las secciones que involucren código, no hay evaluaciones parciales, ya que el algoritmo debe devolver exactamente lo que se pide, de otra forma la respuesta es incorrecta.

En cada pregunta debes ser tan explícito como la calificación que desees obtener. En caso que tu justificación o desarrollo esté incompleto, también lo será la calificación asignada.

##TODA PREGUNTA DE CODIGO DEBE SER VALIDADA CON SU RESPECTIVA PRUEBA QUE MUESTRE LO QUE SE PIDE.

## (2 Puntos) Mínimos Cuadrados (TEORICA)

* Los siguientes datos (reales) corresponden al comportamiento del peso frente al dólar del inicio del año, al mes de Mayo del año en 2017.


Mes   Ene   Feb   Mar   Abr 


       $20.73 20.77 19.90 18.73$


(1 punto) Emplea el gradiente igualado a cero $(∇F(α,β) = 0)$ para expresar el sistema de ecuaciones normales de la tabla 1 en la forma $Ax = b$.
(0.25 puntos) Utiliza el teorema de las ecuaciones normales visto en clase, para expresar el sistema de ecuaciones normales en la forma $Ax = b$.
Verifica que ambas expresiones tengan la misma solución x.
(0.25 puntos) Recta que minimiza la función de costos
(0.25 Puntos) Encuentre la recta que mejor se ajusta a estos datos.
(0.25 Puntos) Trata de dar una estimación del precio del dólar para los meses de Mayo y Junio del mismo año. Para el mes de Mayo x = 5 y para Junio x = 6

HINT: utuliza el código de la [presentación](https://github.com/jugernaut/MACTI-analisisnumerico/blob/main/05_MinimosCuadrados/01_MinimosCuadrados/teacher/Minimos_cuadrados.ipynb) de mínimos cuadrados para validar tus resultados.

## (1 Punto) Error cuadrático medio (PRACTICA)

(0.5 puntos) Función de costos: define la función que calcule el error cuadrático medio (RMS) entre $u$ y $ua$, donde $ua$ es una lista de valores aproximados y $u$ es la lista de valores exactos.

In [3]:
def RMS(ua, u):
    suma_de_cuadrados = 0
    n = len(u)
    for i in range(n):
        suma_de_cuadrados += (u[i] - ua[i])**2
    rmse = (suma_de_cuadrados / n)**0.5
    return rmse

In [4]:
import numpy as np
#Validacion
u = np.array([1, 2, 3, 4, 5])
ua = np.array([1.1, 1.9, 3.2, 4.1, 4.9])

error = RMS(u, ua)
print(f'El error cuadrático medio es: {error}')


El error cuadrático medio es: 0.12649110640673514


## (2 Puntos) Diferencias Finítas (TEORICO)

Muestre el desarrollo para llegar a las siguientes aproximaciones de la segunda
derivada.
$$f′′(x)≈ f(x)−2f(x+△)+f(x+2△) △2$$
$$f′′(x)≈ f(x+2△)−2f(x)+f(x−2△) 4△2$$
$$f′′(x)≈ f(x+2△)−f(x+△)−f(x−△)+f(x−2△)$$

## (5 Puntos) NUMBA (PRACTICA)

Escribe el código en python (usando NUMBA) para las siguientes funciones


In [5]:
from numba import njit, prange
@njit(parallel=True)
# (1 Punto)
# funcion que realiza el producto punto en paralelo
# entre los vectores u y v
def productoPunto(u, v):
  resultado = 0
  for i in prange(len(u)):
       resultado += u[i] * v[i]
  return resultado


In [8]:
u = np.array([1, 2, 3, 4, 5])
ua = np.array([1.1, 1.9, 3.2, 4.1, 4.9])

productoPunto(u,ua)

55.4

In [31]:
# (1 Punto)
# funcion que realiza la suma de 2 matrices en paralelo
# usando GPU's

from numba import cuda
import numpy as np

@cuda.jit
def kernel_suma_matrices(A, B, C):
    fila, columna = cuda.grid(2)
    if fila < C.shape[0] and columna < C.shape[1]:
        C[fila, columna] = A[fila, columna] + B[fila, columna]

def sumaMatrices(mat1, mat2):
    # Verificar que las dimensiones de las matrices son iguales
    if mat1.shape != mat2.shape:
        raise ValueError("Las dimensiones de las matrices deben ser iguales")

    # Crear la matriz de resultado
    C = np.zeros_like(mat1)

    # Configurar el número de threads y bloques
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(mat1.shape[0] / threadsperblock[1]))
    blockspergrid_y = int(np.ceil(mat1.shape[1] / threadsperblock[0]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Transferir los datos a la GPU
    d_mat1 = cuda.to_device(mat1)
    d_mat2 = cuda.to_device(mat2)
    d_C = cuda.to_device(C)

    # Ejecutar el kernel
    kernel_suma_matrices[blockspergrid, threadsperblock](d_mat1, d_mat2, d_C)

    # Transferir el resultado de vuelta al host
    C = d_C.copy_to_host()

    return C


In [33]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
B = np.array([[10, 11, 12], [13, 14, 15], [16, 17, 18]])

C = sumaMatrices(A, B)
print(f'La suma de las matrices es:\n{C}')

La suma de las matrices es:
[[11 13 15]
 [17 19 21]
 [23 25 27]]


In [23]:
# (2 Puntos)
# funcion que realiza la multiplicacion matriz vector
# usando GPU's

from numba import cuda
import numpy as np
import math

@cuda.jit
def multiplicar_matriz_vector_kernel(d_matriz, d_vector, d_resultado):
    fila = cuda.grid(1)
    if fila < d_resultado.size:
        temp = 0.
        for i in range(d_vector.size):
            temp += d_matriz[fila, i] * d_vector[i]
        d_resultado[fila] = temp

def matrizVector(matriz, vector):
    filas = matriz.shape[0]
    columnas = vector.shape[0]

    # Transferir los datos a la GPU
    d_matriz = cuda.to_device(matriz)
    d_vector = cuda.to_device(vector)
    d_resultado = cuda.device_array((filas,))

    # Configurar la cantidad de threads
    threadsperblock = 256
    blockspergrid = (d_resultado.size + (threadsperblock - 1)) // threadsperblock

    # Ejecutar el cálculo
    multiplicar_matriz_vector_kernel[blockspergrid, threadsperblock](d_matriz, d_vector, d_resultado)

    # Devolver el resultado al host
    return d_resultado.copy_to_host()


In [26]:
A =np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
v =np.array([2, 3, 4])

resultado = matrizVector(A, v)
print(f'El resultado de la multiplicación de la matriz por el vector es: {resultado}')


El resultado de la multiplicación de la matriz por el vector es: [20. 47. 74.]


In [35]:
# (2 Puntos)
# funcion que realiza la multiplicacion de matrices
# usando GPU's

from numba import cuda
import numpy as np

@cuda.jit
def kernel_multiplicacion_matrices(mat1, mat2, C):
    fila, columna = cuda.grid(2)
    if fila < C.shape[0] and columna < C.shape[1]:
        tmp = 0.
        for k in range(mat1.shape[1]):
            tmp += mat1[fila, k] * mat2[k, columna]
        C[fila, columna] = tmp


def multiplicacionMatrices(mat1, mat2):
    # Verificar que las dimensiones de las matrices son compatibles para multiplicación
    if mat1.shape[1] != mat2.shape[0]:
        raise ValueError("El número de columnas de la primera matriz debe ser igual al número de filas de la segunda matriz")

    # Crear la matriz de resultado
    C = np.zeros((mat1.shape[0], mat2.shape[1]))

    # Configurar el número de threads y bloques
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(mat1.shape[0] / threadsperblock[1]))
    blockspergrid_y = int(np.ceil(mat2.shape[1] / threadsperblock[0]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Transferir los datos a la GPU
    d_mat1 = cuda.to_device(mat1)
    d_mat2 = cuda.to_device(mat2)
    d_C = cuda.to_device(C)

    # Ejecutar el kernel
    kernel_multiplicacion_matrices[blockspergrid, threadsperblock](d_mat1, d_mat2, d_C)

    # Transferir el resultado de vuelta al host
    C = d_C.copy_to_host()

    return C

In [38]:
mat1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float32)
mat2 = np.array([[10, 11, 12], [13, 14, 15], [16, 17, 18]], dtype=np.float32)

C = multiplicacionMatrices(mat1, mat2)
print(f'El resultado de la multiplicación de las matrices es: \n{C}')


El resultado de la multiplicación de las matrices es: 
[[ 84.  90.  96.]
 [201. 216. 231.]
 [318. 342. 366.]]


## (1 Punto) Cuadratura Gaussiana (TEORICA)
Muestra el desarrollo para encontrar los pesos y putos de Gauss $(ci,xi)$ para la cuadratura Gaussiana con 3 puntos. Y explica que cómo se puede aplicar dicha cuadratura en una integral definida sin importar el rango en el cual esta sea evaluada.

## Método de Monte Carlo (TEORICA)
(1 Punto) Resuelve la siguiente integral de forma analítica y mediante el método de Monte Carlo aproxima la solución para N=1000, N=10000, N=100000. Crea una tabla donde se muestre la comparación de los resultados, así como su porcentaje de error.

$$\intop_{1}^{5}\intop_{1}^{4}\intop_{2}^{3}\,(x^{2}+y^{2}+z^{2})\,dx\,dy\,dz$$

HINT: Puedes usar el código de MonteCarlo para crear la tabla.

In [64]:
import numpy as np 

'''Integracion numerica de Monte Carlo
f: funcion a integrar
N: numero de valores aleatorios
a,b,c,d: limites de integracion
'''
def MonteCarloTriple(f,N,a,b,c,d,g,h):
    # area a integrar
    dim = (b-a)*(d-c)*(h-g)
    # valores aleatorios para								
    a1 = np.random.uniform(size=N)						
    a2 = np.random.uniform(size=N)
    a3 = np.random.uniform(size=N)
    # version alternativa
    #a1 = np.random.uniform(a,b,size=N)
    #a2 = np.random.uniform(c,d,size=N)
    # acumulador temporal
    temp = 0
    # ciclo para realizar la suma de MC											
    for i in range(N):
        # valores recorridos
        xr = a1[i]*(b-a)+a
        yr = a2[i]*(d-c)+c
        zr = a3[i]*(h-g)+g
        # version alternativa
        #temp += fun(a1[i],a2[i])
        # evaluacion
        temp += f(xr,yr, zr)								
    # aproximacion mediante MC
    return dim*temp/N

# (6 Puntos) Diferencias Finítas (PRACTICO)

En la siguiente celda se muestra el código para resolver la ecuación que modela la transferencia de calor mediante diferencias finítas haciendo uso del esquema implícito.

In [39]:
import numpy as np
from IPython.html.widgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from time import time

"""
Método que aproxima la ecuación de calor mediante el esquema explícito
K, difusión de calor
b, condición inicial T(0,0)
c, condición de frontera T(Nx,t)
a, valor toman todoas las x's en el tiempo cero
Nx, variable espacial
Nt, variable temporal
delt, cambios en el tiempo
"""

def ec_calor_exp(K,a,b,c,Nx,delt,Nt,xmax):
    dx=xmax/Nx #Cambios de x                       
    D=K*delt/(dx*dx) #Constante de estabilidad
    T=np.zeros((Nx,Nt)) #Nx renglones, Nt columnas
    T[0][0]=b #Condición inicial

    dx = xmax/Nx #Cambios de x                       
    D = K*delt/(dx*dx) #Constante de estabilidad
    
    Temp = np.full((2,Nx), float(a))
    Temp[1][0] = b
    Temp[1][-1] = c

    if (D > 0.5): #Condición de estabilidad
        print("Algoritmo inestable; Terminar") #Termina si no cumple 
    else:
        for n in range(2, Nt+1):
            tempAnt = Temp[-1].copy()
            tempAct = Temp[-1].copy()
            for i in range (1,Nx-1):
                #Cálculo de aproximaciones
                #T[i][n+1]=T[i][n] + D * (T[i+1][n] - 2*T[i][n] + T[i-1][n])
                tempAct[i]=tempAnt[i] + D * (tempAnt[i+1] - 2*tempAnt[i] + tempAnt[i-1]) 
            Temp = np.vstack([Temp, tempAct])    
        tiempo_fin=time() #Tiempo de ejecución después de terminar el método
    return Temp     #Cada columna representa las temperaturas en el tiempo n  

"""
Método que gráfica la temperatura de cada punto a través del tiempo 
"""
def grafica(mat, xmax, Nx, n):
  disc=np.linspace(0,xmax,Nx) #Discretización
  #Grafica los puntos de cada columna de la matriz mat
  plt.plot(disc, mat[n],'r')
  plt.scatter(disc, mat[n],c='#8c564b')        
  plt.title("Diferencias Finitas Ec. de Calor, Tiempo "+str(n))
  plt.show()

#Declaramos las variables 
K=np.e**(-2)
a=323
b=273
c=373
Nx=25 
delt=.001
Nt=100
xmax=0.5

#llama a la implementación del esquema explícito
explicito=ec_calor_exp(K,a,b,c,Nx,delt,Nt,xmax)
@widgets.interact(t=(0, Nt, 1))
def f2(t):
    return grafica(explicito,xmax,Nx,t)

interactive(children=(IntSlider(value=50, description='t'), Output()), _dom_classes=('widget-interact',))

## (3 Puntos) Implícito

Toma como base este algoritmo, resuelve la misma ecuación usando diferencias finitas pero con el **esquema implícito**.

In [46]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

def ec_calor_imp(K,a,b,c,Nx,delt,Nt,xmax):
    dx = xmax/Nx 
    D = K*delt/(dx*dx) 
    T = np.zeros((Nx,Nt)) 

    T[:,0] = a
    T[0,:] = b
    T[-1,:] = c

    for n in range(1,Nt):
        # Crear copia de la temperatura actual
        T_new = T[:,n-1].copy()

        # Iterar hasta alcanzar la convergencia
        while True:
            T_old = T_new.copy()

            # Resolver para las temperaturas internas
            for i in range(1, Nx - 1):
                T_new[i] = (T[i, n-1] + D * (T_old[i - 1] + T_old[i + 1])) / (1 + 2 * D)

            # Comprobar convergencia
            if np.allclose(T_old, T_new, rtol=1e-6):
                break

        T[:,n] = T_new

    return T

def grafica(mat, xmax, Nx, n):
    disc = np.linspace(0, xmax, Nx)
    plt.plot(disc, mat[:,n],'r')
    plt.scatter(disc, mat[:,n],c='#8c564b')        
    plt.title("Diferencias Finitas Ec. de Calor, Tiempo "+str(n))
    plt.show()

K = np.e**(-2)
a = 323
b = 273
c = 373
Nx = 25 
delt = .001
Nt = 100
xmax = 0.5

implicito = ec_calor_imp(K,a,b,c,Nx,delt,Nt,xmax)
@widgets.interact(t=(0, Nt, 1))
def f2(t):
    return grafica(implicito,xmax,Nx,t)                                 


interactive(children=(IntSlider(value=50, description='t'), Output()), _dom_classes=('widget-interact',))

## (3 Puntos) Paralelo

Paraleliza (Usando Numba y CUDA) el código de la celda superior.

In [45]:
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def compute_next_timestep(T, D, Nx):
    T_new = T.copy()
    for i in prange(1, Nx - 1):
        T_new[i] = (T[i] + D * (T[i - 1] - 2 * T[i] + T[i + 1])) / (1 + 2 * D)
    return T_new

def ec_calor_imp(K, a, b, c, Nx, delt, Nt, xmax):
    dx = xmax / Nx
    D = K * delt / dx**2
    T = np.zeros((Nt, Nx))

    # condiciones iniciales y de frontera
    T[0, :] = a
    T[:, 0] = b
    T[:, -1] = c

    for t in range(Nt - 1):
        T[t + 1, :] = compute_next_timestep(T[t, :], D, Nx)

    return T

def grafica(mat, xmax, Nx, n):
    disc = np.linspace(0, xmax, Nx)
    plt.plot(disc, mat[n, :],'r')
    plt.scatter(disc, mat[n, :],c='#8c564b')        
    plt.title("Diferencias Finitas Ec. de Calor, Tiempo "+str(n))
    plt.show()

K = np.e**(-2)
a = 323
b = 273
c = 373
Nx = 25 
delt = .001
Nt = 100
xmax = 0.5

implicito = ec_calor_imp(K,a,b,c,Nx,delt,Nt,xmax)

@widgets.interact(t=(0, Nt - 1, 1))
def f2(t):
    return grafica(implicito,xmax,Nx,t)

interactive(children=(IntSlider(value=49, description='t', max=99), Output()), _dom_classes=('widget-interact'…

# (¿? Puntos) Sección Especial

Esta sección es solo para amenizar al momento de calificar y probablemente subir unas cuantas décimas. Pero por ningún motivo esta sección hará que apruebes el examen si es que no muestras conocimientos.

***SpiderMan***
<center>
<img src="https://media.tenor.com/MK_WlzhpxaoAAAAd/the-last-of-us-the-last-of-us-tv-series.gif" width="650">
</center>  

- ¿Cuál es el nombre cientifico del hongo que desata la pandemia en TLOU?

- ¿Por que Elie es inmune al hongo?

- ¿Qué acontecimiento tragico sucede al inicio de la segunda parte del juego?








# Referencias

*   [Bases de Datos](https://github.com/jugernaut/ManejoDatos/tree/main/Bases_de_Datos)
*   [Estándares](https://colab.research.google.com/github/jugernaut/ManejoDatos/blob/main/EstandaresInformacion/01_EstandaresInformacion.ipynb)